In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import ipywidgets as widgets
from IPython.display import display, HTML
from arch import arch_model
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

In [ ]:
def fit_arma_garch(tickers):
    """
    Run ARMA + GARCH analysis for each ticker in the list of selected symbols.
    """

    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    colors = ['#2ca02c', '#1f77b4', '#9467bd', '#d62728']
    
    # Start button
    button = widgets.Button(
        description="Run ARMA–GARCH",
        layout=widgets.Layout(width='200px', height='40px', margin='10px 0 10px 0'),
        style={'font_weight': 'bold', 'font_size': '16px', 'button_color': '#ffb6c1'}
    )

    output = widgets.Output()

    # Unified title
    def show_title(title):
        display(HTML(f"""
            <h3 style="
                text-align:center;
                margin:30px 0 10px 0;
                color:#2c3e50;
                font-weight:bold;
                font-size:25px;
            ">
                {title}
            </h3>
        """))

    # Best ARMA grid search
    def select_best_arma(returns, max_p=5, max_q=5):
        
        best_aic = np.inf
        best_order = None
        best_model = None

        for p in range(max_p + 1):
            for q in range(max_q + 1):
                if p == 0 and q == 0:
                    continue
                try:
                    model = ARIMA(
                        returns,
                        order=(p, 0, q),
                        enforce_stationarity=False,
                        enforce_invertibility=False
                    ).fit()

                    if np.isnan(model.aic) or np.isinf(model.aic):
                        continue

                    if model.aic < best_aic:
                        best_aic = model.aic
                        best_order = (p, q)
                        best_model = model

                except Exception:
                    continue

        return best_model, best_order

    # Best GARCH grid search
    def select_best_garch(residuals, max_p=3, max_q=3):
        best_aic = np.inf
        best_order = None
        best_model = None

        for p in range(1, max_p + 1):
            for q in range(1, max_q + 1):
                try:
                    model = arch_model(
                        residuals,
                        vol="GARCH",
                        p=p,
                        q=q,
                        rescale=False
                    ).fit(disp="off")

                    if model.aic < best_aic:
                        best_aic = model.aic
                        best_order = (p, q)
                        best_model = model

                except Exception:
                    continue

        return best_model, best_order

    # After clicking the button
    def on_button_click(b):
        
        with output:
            output.clear_output()

            show_title(f"\U0001F52C ARMA-GARCH Analysis")

            for i, ticker in enumerate(tickers):
                color_returns = colors[i % len(colors)]
                print(f"\n\U0001F4C8 {ticker} \U0001F4C9")

                df = yf.download(
                    ticker,
                    period="2y",
                    interval="1d",
                    auto_adjust=True,
                    progress=False
                )

                if df.empty:
                    print(f"\U0000274C No data downloaded for {ticker}. Skipping this stock.")
                    continue

                prices = df["Close"]
                log_returns = np.log(prices).diff().dropna()
                log_returns = log_returns.asfreq("D").fillna(0)

                # ADF test
                adf_stat, p_value, *_ = adfuller(log_returns, maxlag=1)
                is_stationary = p_value < 0.05

                # ARMA
                arma_model, order = select_best_arma(log_returns)

                if arma_model is None:
                    print(f"\U0000274C ARMA model could not be estimated.")
                    continue

                p, q = order
                residuals = arma_model.resid

                # GARCH
                garch_model, g_order = select_best_garch(residuals)

                if garch_model is None:
                    print(f"\U0000274C GARCH model could not be estimated.")
                    continue

                # Volatility
                gp, gq = g_order
                cond_vol = garch_model.conditional_volatility
                cond_vol.index = log_returns.index[-len(cond_vol):]

                # ARMA fitted mean
                fitted_mean = arma_model.fittedvalues
                fitted_mean = fitted_mean.iloc[-len(cond_vol):]

                # Volatility confidence bands (95%)
                upper = fitted_mean + 1.96 * cond_vol
                lower = fitted_mean - 1.96 * cond_vol

                status = "Stationary \U00002705" if is_stationary else "NOT stationary - ARMA-GARCH analysis invalid \U0000274C"
                print(
                    f"{ticker}: {status}\n"
                    f"ADF Statistic: {adf_stat:.4f}, p-value: {p_value:.4f}\n"
                    f"ARMA({p},{q}), GARCH({gp},{gq})"
                )
                
                # Graph
                plt.figure(figsize=(12, 5))
                plt.plot(log_returns, label="Log Returns", alpha=0.6, color=color_returns)
                plt.plot(fitted_mean, label="ARMA Mean", linewidth=2, color="orange")
                plt.fill_between(fitted_mean.index, lower, upper, color="red", alpha=0.20, label="95% Confidence Bound")
                plt.title(f"{ticker}: ARMA({p},{q}) + GARCH({gp},{gq})")
                plt.legend()
                plt.show()

    button.on_click(on_button_click)

    display(widgets.VBox([button, output],
                         layout=widgets.Layout(align_items="center")))
